# Hierarchical Clustering and Dendrograms

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Explain agglomerative (bottom-up) and divisive (top-down) hierarchical clustering
2. Understand linkage types: single, complete, average, and Ward
3. Create and interpret dendrograms using `scipy.cluster.hierarchy`
4. Use `AgglomerativeClustering` from scikit-learn
5. Compare hierarchical clustering results with K-Means on the same data
6. Decide when to use hierarchical clustering vs. K-Means

## Prerequisites

- Notebook 02 (K-Means and Cluster Evaluation)
- Familiarity with distance metrics (Notebook 01)
- NumPy, Matplotlib basics

## Table of Contents

1. [Theory: Agglomerative vs. Divisive](#1-theory-agglomerative-vs-divisive)
2. [Linkage Types](#2-linkage-types)
3. [Dendrograms](#3-dendrograms)
4. [Cutting the Dendrogram](#4-cutting-the-dendrogram)
5. [AgglomerativeClustering in scikit-learn](#5-agglomerativeclustering-in-scikit-learn)
6. [Comparison with K-Means](#6-comparison-with-k-means)
7. [When to Use Hierarchical vs. K-Means](#7-when-to-use-hierarchical-vs-k-means)
8. [Common Mistakes](#8-common-mistakes)
9. [Exercise](#9-exercise)

---

## 1. Theory: Agglomerative vs. Divisive

Hierarchical clustering builds a **tree of clusters** (dendrogram) rather than producing a flat partition.

**Agglomerative (bottom-up):**
1. Start with each point as its own cluster.
2. Merge the two closest clusters.
3. Repeat until all points are in one cluster.

**Divisive (top-down):**
1. Start with all points in one cluster.
2. Split a cluster into two.
3. Repeat until each point is its own cluster.

Agglomerative is far more common and is the default in scikit-learn. Divisive is computationally more expensive and rarely used in practice.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)

print("Imports complete.")

In [ ]:
# Generate data
X, y_true = make_blobs(n_samples=200, centers=4, cluster_std=0.6, random_state=42)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y_true, cmap="viridis", s=30, edgecolors="k")
plt.title("Synthetic Blobs (true labels)")
plt.xlabel("Feature 1 (scaled)")
plt.ylabel("Feature 2 (scaled)")
plt.show()

---

## 2. Linkage Types

When merging two clusters, the "distance" between them depends on the **linkage** criterion:

| Linkage | Definition | Behavior |
|---|---|---|
| **Single** | Min distance between any two points in the two clusters | Can produce elongated, "chaining" clusters |
| **Complete** | Max distance between any two points in the two clusters | Produces compact, equally-sized clusters |
| **Average** | Mean distance between all pairs across clusters | Compromise between single and complete |
| **Ward** | Minimizes increase in total within-cluster variance | Produces compact, similarly-sized clusters (most popular) |

In [ ]:
# Compute linkage matrices for each method
linkage_methods = ["single", "complete", "average", "ward"]
linkage_matrices = {}

for method in linkage_methods:
    linkage_matrices[method] = linkage(X_scaled, method=method)

print("Linkage matrices computed for:", linkage_methods)

---

## 3. Dendrograms

A **dendrogram** is a tree diagram showing the order of merges. The y-axis represents the distance (or dissimilarity) at which clusters merge. Taller vertical lines indicate bigger jumps in distance.

**How to read a dendrogram:**
- Leaf nodes at the bottom are individual data points.
- The height at which two branches merge indicates how dissimilar those clusters are.
- A large vertical gap suggests a natural number of clusters (cut there).

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for ax, method in zip(axes.ravel(), linkage_methods):
    dendrogram(
        linkage_matrices[method],
        truncate_mode="lastp",
        p=30,
        leaf_rotation=90,
        leaf_font_size=8,
        ax=ax
    )
    ax.set_title(f"Dendrogram ({method} linkage)")
    ax.set_xlabel("Sample index (or cluster size)")
    ax.set_ylabel("Distance")

plt.tight_layout()
plt.show()
print("Ward linkage typically produces the clearest separation for globular clusters.")

---

## 4. Cutting the Dendrogram

To obtain a flat clustering, we "cut" the dendrogram at a chosen height (distance threshold) or specify the number of clusters. Below we demonstrate both approaches.

In [ ]:
# Full dendrogram with Ward linkage, showing a horizontal cut line
Z_ward = linkage_matrices["ward"]

plt.figure(figsize=(14, 6))
dendrogram(Z_ward, truncate_mode="lastp", p=30, leaf_rotation=90, leaf_font_size=8)
plt.axhline(y=5.0, color="r", linestyle="--", linewidth=2, label="Cut at distance = 5.0")
plt.title("Ward Dendrogram with Cut Line")
plt.xlabel("Sample index (or cluster size)")
plt.ylabel("Distance")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Cut by distance threshold
labels_cut_dist = fcluster(Z_ward, t=5.0, criterion="distance")
print(f"Cutting at distance=5.0 produces {len(np.unique(labels_cut_dist))} clusters")

# Cut by number of clusters
labels_cut_k = fcluster(Z_ward, t=4, criterion="maxclust")
print(f"Cutting at k=4 produces {len(np.unique(labels_cut_k))} clusters")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels_cut_dist, cmap="viridis", s=30, edgecolors="k")
axes[0].set_title(f"Cut by distance=5.0 ({len(np.unique(labels_cut_dist))} clusters)")

axes[1].scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels_cut_k, cmap="viridis", s=30, edgecolors="k")
axes[1].set_title("Cut by maxclust=4")

plt.tight_layout()
plt.show()

---

## 5. AgglomerativeClustering in scikit-learn

scikit-learn provides `AgglomerativeClustering` with a familiar `.fit_predict()` API.

In [ ]:
agg = AgglomerativeClustering(n_clusters=4, linkage="ward")
labels_agg = agg.fit_predict(X_scaled)

sil = silhouette_score(X_scaled, labels_agg)
print(f"AgglomerativeClustering (ward, k=4) Silhouette Score: {sil:.4f}")

plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels_agg, cmap="viridis", s=30, edgecolors="k")
plt.title("AgglomerativeClustering (Ward, k=4)")
plt.xlabel("Feature 1 (scaled)")
plt.ylabel("Feature 2 (scaled)")
plt.show()

In [ ]:
# Compare different linkage types with sklearn
fig, axes = plt.subplots(1, 4, figsize=(20, 4))

for ax, link in zip(axes, ["single", "complete", "average", "ward"]):
    agg_link = AgglomerativeClustering(n_clusters=4, linkage=link)
    labels_link = agg_link.fit_predict(X_scaled)
    sil_link = silhouette_score(X_scaled, labels_link)
    ax.scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels_link, cmap="viridis", s=20, edgecolors="k")
    ax.set_title(f"{link} (sil={sil_link:.3f})")

plt.suptitle("AgglomerativeClustering: Linkage Comparison", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---

## 6. Comparison with K-Means

Let us run both KMeans and AgglomerativeClustering (Ward) on the same data and compare.

In [ ]:
km = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_km = km.fit_predict(X_scaled)
sil_km = silhouette_score(X_scaled, labels_km)

agg_ward = AgglomerativeClustering(n_clusters=4, linkage="ward")
labels_ward = agg_ward.fit_predict(X_scaled)
sil_ward = silhouette_score(X_scaled, labels_ward)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].scatter(X_scaled[:, 0], X_scaled[:, 1], c=y_true, cmap="viridis", s=30, edgecolors="k")
axes[0].set_title("True Labels")

axes[1].scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels_km, cmap="viridis", s=30, edgecolors="k")
axes[1].set_title(f"KMeans (sil={sil_km:.3f})")

axes[2].scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels_ward, cmap="viridis", s=30, edgecolors="k")
axes[2].set_title(f"Agglomerative Ward (sil={sil_ward:.3f})")

plt.tight_layout()
plt.show()
print("For globular blobs, both methods perform similarly well.")

---

## 7. When to Use Hierarchical vs. K-Means

| Criterion | K-Means | Hierarchical |
|---|---|---|
| **Speed** | Fast (O(nk) per iteration) | Slow (O(n^2) or O(n^3)) |
| **Scalability** | Handles large datasets well | Best for small to medium datasets |
| **Number of clusters** | Must specify k in advance | Can explore different k via dendrogram |
| **Cluster shape** | Assumes spherical | Ward assumes spherical; single linkage finds elongated shapes |
| **Interpretability** | Centroids are easy to understand | Dendrogram shows hierarchy of relationships |
| **Determinism** | Non-deterministic (depends on init) | Deterministic |

---

## 8. Common Mistakes

| Mistake | Why It Matters |
|---|---|
| **Using single linkage on noisy data** | Single linkage is prone to the "chaining effect" -- noise can bridge two separate clusters into one. Use Ward or complete linkage instead. |
| **Ignoring the dendrogram scale** | The y-axis of a dendrogram shows distances. Large jumps indicate natural cluster boundaries. Always inspect the scale. |
| **Not scaling features** | Like K-Means, hierarchical clustering relies on distances. Always scale features first. |
| **Running on very large datasets** | Hierarchical clustering is O(n^2) in memory and O(n^3) in time. For large n, use K-Means or mini-batch KMeans. |
| **Choosing linkage blindly** | Different linkage types produce very different results. Always compare at least Ward vs. complete vs. average. |

---

## 9. Exercise

**Task:** Generate data using `make_blobs(n_samples=150, centers=3, cluster_std=1.0, random_state=42)`. Scale the data. Create a Ward dendrogram and visually identify the best number of clusters. Then run `AgglomerativeClustering` with that k, compute the silhouette score, and visualize the result.

Bonus: Compare the silhouette scores of single, complete, average, and Ward linkage for the same k.

In [ ]:
# YOUR CODE HERE

# 1. Generate blobs and scale
# 2. Compute Ward linkage and plot dendrogram
# 3. Choose k from the dendrogram
# 4. Run AgglomerativeClustering with that k
# 5. Compute silhouette score and visualize